# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO

For this exercise, we will use the well-known MNIST digit data. To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
#from tensorflow import keras
#from tensorflow.keras.datasets import mnist
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
#from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras import backend as K

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [4]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [5]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

11490434/11490434 [==============================] - 5s 0us/step


In [8]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [9]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [10]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [11]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [12]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 32s 134ms/step - loss: 1.6192 - accuracy: 0.1982 - val_loss: 1.6074 - val_accuracy: 0.2115
Epoch 2/5
230/230 [==============================] - 28s 120ms/step - loss: 1.6009 - accuracy: 0.2433 - val_loss: 1.5869 - val_accuracy: 0.3205
Epoch 3/5
230/230 [==============================] - 26s 112ms/step - loss: 1.5831 - accuracy: 0.2972 - val_loss: 1.5657 - val_accuracy: 0.4349
Epoch 4/5
230/230 [==============================] - 28s 120ms/step - loss: 1.5629 - accuracy: 0.3520 - val_loss: 1.5429 - val_accuracy: 0.5017
Epoch 5/5
230/230 [==============================] - 29s 125ms/step - loss: 1.5408 - accuracy: 0.4053 - val_loss: 1.5178 - val_accuracy: 0.5357
Training time: 0:02:21.397355
Test score: 1.5178062915802002
Test accuracy: 0.5356922149658203


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [13]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [15]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 11s 44ms/step - loss: 1.6043 - accuracy: 0.2398 - val_loss: 1.5783 - val_accuracy: 0.3664
Epoch 2/5
240/240 [==============================] - 11s 45ms/step - loss: 1.5686 - accuracy: 0.3280 - val_loss: 1.5422 - val_accuracy: 0.4365
Epoch 3/5
240/240 [==============================] - 11s 48ms/step - loss: 1.5363 - accuracy: 0.3966 - val_loss: 1.5082 - val_accuracy: 0.4982
Epoch 4/5
240/240 [==============================] - 12s 50ms/step - loss: 1.5061 - accuracy: 0.4522 - val_loss: 1.4753 - val_accuracy: 0.5709
Epoch 5/5
240/240 [==============================] - 13s 54ms/step - loss: 1.4754 - accuracy: 0.5060 - val_loss: 1.4424 - val_accuracy: 0.6416
Training time: 0:00:58.555103
Test score: 1.442378282546997
Test accuracy: 0.6415644884109497


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.

In [16]:
# Create layers and define the model as above
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]
model2 = Sequential(feature_layers2 + classification_layers2)
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [17]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 37s 152ms/step - loss: 1.6096 - accuracy: 0.2137 - val_loss: 1.5738 - val_accuracy: 0.3520
Epoch 2/5
240/240 [==============================] - 38s 159ms/step - loss: 1.5651 - accuracy: 0.3202 - val_loss: 1.5266 - val_accuracy: 0.6425
Epoch 3/5
240/240 [==============================] - 41s 173ms/step - loss: 1.5217 - accuracy: 0.4276 - val_loss: 1.4765 - val_accuracy: 0.7881
Epoch 4/5
240/240 [==============================] - 38s 158ms/step - loss: 1.4729 - accuracy: 0.5305 - val_loss: 1.4205 - val_accuracy: 0.8430
Epoch 5/5
240/240 [==============================] - 37s 153ms/step - loss: 1.4170 - accuracy: 0.6140 - val_loss: 1.3548 - val_accuracy: 0.8718
Training time: 0:03:11.717879
Test score: 1.3547980785369873
Test accuracy: 0.8717649579048157


In [18]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [19]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [20]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 16s 69ms/step - loss: 1.5868 - accuracy: 0.2649 - val_loss: 1.5507 - val_accuracy: 0.3779
Epoch 2/5
230/230 [==============================] - 17s 75ms/step - loss: 1.5510 - accuracy: 0.3144 - val_loss: 1.5114 - val_accuracy: 0.4557
Epoch 3/5
230/230 [==============================] - 12s 51ms/step - loss: 1.5146 - accuracy: 0.3800 - val_loss: 1.4724 - val_accuracy: 0.5591
Epoch 4/5
230/230 [==============================] - 14s 62ms/step - loss: 1.4779 - accuracy: 0.4467 - val_loss: 1.4339 - val_accuracy: 0.6351
Epoch 5/5
230/230 [==============================] - 14s 61ms/step - loss: 1.4439 - accuracy: 0.5012 - val_loss: 1.3958 - val_accuracy: 0.6867
Training time: 0:01:13.543973
Test score: 1.3958033323287964
Test accuracy: 0.6866899728775024


---
### Machine Learning Foundation (C) 2020 IBM Corporation